## IBKR Functions
Comprehensive class for retrieving historical data, live data, option chains, and option data from Interactive Brokers using the official ibapi library.

In [1]:
from ibapi.client import EClient
from ibapi.wrapper import EWrapper
from ibapi.contract import Contract
from ibapi.order import Order
import threading
import time
from datetime import datetime
import pandas as pd

class IBKR_Functions(EWrapper, EClient):
    """
    Comprehensive IBKR API client class for retrieving live data, historical data, 
    option chains, and option data using the official ibapi library.
    
    This class provides user-friendly methods to interact with Interactive Brokers API
    with all output in pandas DataFrame format for easy analysis.
    
    Features:
    - Live market data streaming
    - Historical data retrieval
    - Option chain discovery
    - Option-specific data
    - Automatic connection management
    - Thread-safe operations
    - Error handling and logging
    
    Example:
        client = IBKR_Functions(host='127.0.0.1', port=7496, client_id=1)
        client.connect()
        hist_data = client.get_historical_data('AAPL', duration='1 M', bar_size='1 day')
        client.disconnect()
    """
    
    def __init__(self, host='127.0.0.1', port=7497, client_id=1):
        """
        Initialize the IBKR Functions client.
        
        Args:
            host (str): Host address for IBKR TWS/Gateway (default: '127.0.0.1' for localhost)
            port (int): Port number 
                       - 7497: TWS Paper Trading
                       - 7496: TWS Live Trading
                       - 4001: IB Gateway Paper Trading
                       - 4000: IB Gateway Live Trading
            client_id (int): Unique client ID (0-32). Use 0 for master client.
        """
        # Initialize parent classes
        EClient.__init__(self, self)
        
        # Connection parameters
        self.host = host
        self.port = port
        self.client_id = client_id
        
        # Request ID management
        self.next_order_id = None
        self._req_id_counter = 1000  # Start from 1000 to avoid conflicts
        
        # Data storage dictionaries
        self.historical_data = {}  # Stores historical bars
        self.live_data = {}  # Stores streaming market data
        self.option_chains = {}  # Stores option chain details
        self.contract_details_data = {}  # Stores contract specifications
        self.option_greeks = {}  # Stores option greeks
        
        # Event flags for synchronization
        self.data_received_event = threading.Event()
        self.connected_event = threading.Event()
        self.contract_details_end_event = threading.Event()
        
        # Error tracking
        self.last_error = None
        
        # Thread for running the client loop
        self.api_thread = None
        
    # ==================== CONNECTION MANAGEMENT ====================
    
    def connect(self):
        """
        Establish connection to IBKR TWS or IB Gateway.
        
        Returns:
            bool: True if connection successful, False otherwise
        """
        try:
            # Connect to IBKR
            EClient.connect(self, self.host, self.port, self.client_id)
            
            # Start the socket in a separate thread
            self.api_thread = threading.Thread(target=self.run, daemon=True)
            self.api_thread.start()
            
            # Wait for connection confirmation (nextValidId callback)
            if self.connected_event.wait(timeout=10):
                print(f"✓ Successfully connected to IBKR on {self.host}:{self.port} (Client ID: {self.client_id})")
                time.sleep(0.5)  # Brief pause to ensure stability
                return True
            else:
                print("✗ Connection timeout - Please ensure TWS/Gateway is running")
                return False
                
        except Exception as e:
            print(f"✗ Connection error: {str(e)}")
            return False
    
    def disconnect(self):
        """
        Disconnect from IBKR and clean up resources.
        """
        try:
            EClient.disconnect(self)
            print("✓ Disconnected from IBKR")
        except Exception as e:
            print(f"✗ Disconnection error: {str(e)}")
    
    def is_connected(self):
        """
        Check if client is currently connected to IBKR.
        
        Returns:
            bool: True if connected, False otherwise
        """
        return self.isConnected()
    
    # ==================== CALLBACK METHODS (EWrapper) ====================
    
    def error(self, reqId, errorCode, errorString, advancedOrderReject=""):
        """
        Handle error messages from IBKR.
        
        Error codes:
        - 502: Could not connect to TWS
        - 200: No security definition found
        - 162: Historical market data service error
        """
        # Store last error
        self.last_error = {'reqId': reqId, 'code': errorCode, 'msg': errorString}
        
        # Filter out informational messages (codes >= 2000)
        if errorCode >= 2000:
            if errorCode in [2104, 2106, 2158]:  # Data farm connection messages
                print(f"ℹ Info: {errorString}")
        else:
            print(f"⚠ Error {errorCode} (ReqId: {reqId}): {errorString}")
    
    def nextValidId(self, orderId):
        """
        Callback when connection is established. Receives the next valid order ID.
        
        Args:
            orderId (int): Next valid order ID
        """
        super().nextValidId(orderId)
        self.next_order_id = orderId
        self._req_id_counter = orderId + 1000  # Offset for request IDs
        self.connected_event.set()
        print(f"ℹ Connection established - Next valid order ID: {orderId}")
    
    def _get_next_req_id(self):
        """
        Generate next unique request ID.
        
        Returns:
            int: Unique request ID
        """
        req_id = self._req_id_counter
        self._req_id_counter += 1
        return req_id
    
    # ==================== HISTORICAL DATA METHODS ====================
    
    def historicalData(self, reqId, bar):
        """
        Callback for receiving historical data bars.
        
        Args:
            reqId (int): Request identifier
            bar (BarData): Historical bar data
        """
        if reqId not in self.historical_data:
            self.historical_data[reqId] = []
        
        # Store bar data as dictionary
        self.historical_data[reqId].append({
            'date': bar.date,
            'open': bar.open,
            'high': bar.high,
            'low': bar.low,
            'close': bar.close,
            'volume': bar.volume,
            'average': bar.average,
            'barCount': bar.barCount
        })
    
    def historicalDataEnd(self, reqId, start, end):
        """
        Callback when all historical data has been received.
        
        Args:
            reqId (int): Request identifier
            start (str): Start date of data
            end (str): End date of data
        """
        print(f"✓ Historical data received for request {reqId} ({start} to {end})")
        self.data_received_event.set()
    
    def get_historical_data(self, symbol, duration='1 Y', bar_size='1 day', 
                           what_to_show='TRADES', sec_type='STK', exchange='SMART', 
                           currency='USD', end_datetime='', use_rth=True, timeout=30):
        """
        Retrieve historical market data for a security.
        
        Args:
            symbol (str): Security symbol (e.g., 'AAPL', 'EUR', 'CL')
            duration (str): How far back to retrieve data
                           Valid units: S (seconds), D (days), W (weeks), M (months), Y (years)
                           Examples: '1 Y', '6 M', '1 W', '3 D'
            bar_size (str): Granularity of bars
                           Valid sizes: '1 secs', '5 secs', '10 secs', '15 secs', '30 secs',
                                       '1 min', '2 mins', '3 mins', '5 mins', '10 mins', '15 mins', '20 mins', '30 mins',
                                       '1 hour', '2 hours', '3 hours', '4 hours', '8 hours',
                                       '1 day', '1 week', '1 month'
            what_to_show (str): Type of data to retrieve
                               'TRADES', 'MIDPOINT', 'BID', 'ASK', 'BID_ASK', 
                               'HISTORICAL_VOLATILITY', 'OPTION_IMPLIED_VOLATILITY',
                               'ADJUSTED_LAST' (for stocks - adjusted for splits/dividends)
            sec_type (str): Security type ('STK', 'OPT', 'FUT', 'CASH', 'IND', 'CFD', 'BOND')
            exchange (str): Exchange ('SMART' for smart routing, or specific exchange)
            currency (str): Currency (e.g., 'USD', 'EUR', 'GBP')
            end_datetime (str): End date/time in format 'yyyyMMdd HH:mm:ss' (empty string for now)
            use_rth (bool): Use Regular Trading Hours only (True) or include extended hours (False)
            timeout (int): Maximum seconds to wait for data
            
        Returns:
            pandas.DataFrame: Historical data with columns: date, open, high, low, close, volume, average, barCount
                             Returns empty DataFrame if request fails
        
        Examples:
            # Get 1 year of daily stock data
            df = client.get_historical_data('AAPL', duration='1 Y', bar_size='1 day')
            
            # Get 1 week of hourly futures data
            df = client.get_historical_data('CL', duration='1 W', bar_size='1 hour', 
                                           sec_type='FUT', exchange='NYMEX')
            
            # Get 1 month of 5-minute forex data
            df = client.get_historical_data('EUR', duration='1 M', bar_size='5 mins', 
                                           sec_type='CASH', currency='USD', exchange='IDEALPRO')
        """
        if not self.is_connected():
            print("✗ Not connected to IBKR. Call connect() first.")
            return pd.DataFrame()
        
        # Create contract object
        contract = Contract()
        contract.symbol = symbol
        contract.secType = sec_type
        contract.exchange = exchange
        contract.currency = currency
        
        # Generate unique request ID
        req_id = self._get_next_req_id()
        
        # Clear any previous data for this request
        self.data_received_event.clear()
        if req_id in self.historical_data:
            del self.historical_data[req_id]
        
        print(f"→ Requesting historical data for {symbol}...")
        
        # Request historical data
        # Parameters: reqId, contract, endDateTime, durationStr, barSizeSetting, 
        #             whatToShow, useRTH, formatDate, keepUpToDate, chartOptions
        self.reqHistoricalData(
            req_id, contract, end_datetime, duration, bar_size, 
            what_to_show, int(use_rth), 1, False, []
        )
        
        # Wait for data to be received
        if self.data_received_event.wait(timeout=timeout):
            if req_id in self.historical_data and len(self.historical_data[req_id]) > 0:
                df = pd.DataFrame(self.historical_data[req_id])
                print(f"✓ Retrieved {len(df)} bars of historical data")
                return df
            else:
                print("✗ No historical data received")
                return pd.DataFrame()
        else:
            print(f"✗ Request timeout after {timeout} seconds")
            return pd.DataFrame()
    
    # ==================== LIVE DATA METHODS ====================
    
    def tickPrice(self, reqId, tickType, price, attrib):
        """
        Callback for receiving real-time price updates.
        
        Args:
            reqId (int): Request identifier
            tickType (int): Type of tick (1=bid, 2=ask, 4=last, 6=high, 7=low, 9=close)
            price (float): Price value
            attrib (TickAttrib): Additional attributes
        """
        if reqId not in self.live_data:
            self.live_data[reqId] = {'timestamp': datetime.now()}
        
        # Map tick types to field names
        tick_map = {
            1: 'bid', 2: 'ask', 4: 'last', 6: 'high', 7: 'low', 9: 'close',
            14: 'open', 15: 'low_13_week', 16: 'high_13_week',
            17: 'low_26_week', 18: 'high_26_week', 19: 'low_52_week', 20: 'high_52_week'
        }
        
        if tickType in tick_map:
            self.live_data[reqId][tick_map[tickType]] = price
            self.live_data[reqId]['timestamp'] = datetime.now()
    
    def tickSize(self, reqId, tickType, size):
        """
        Callback for receiving real-time size updates.
        
        Args:
            reqId (int): Request identifier
            tickType (int): Type of tick (0=bid_size, 3=ask_size, 5=last_size, 8=volume)
            size (int): Size value
        """
        if reqId not in self.live_data:
            self.live_data[reqId] = {'timestamp': datetime.now()}
        
        # Map tick types to field names
        size_map = {
            0: 'bid_size', 3: 'ask_size', 5: 'last_size', 8: 'volume',
            21: 'avg_volume'
        }
        
        if tickType in size_map:
            self.live_data[reqId][size_map[tickType]] = size
            self.live_data[reqId]['timestamp'] = datetime.now()
    
    def tickString(self, reqId, tickType, value):
        """
        Callback for receiving string tick data.
        
        Args:
            reqId (int): Request identifier
            tickType (int): Type of tick
            value (str): String value
        """
        if reqId not in self.live_data:
            self.live_data[reqId] = {'timestamp': datetime.now()}
        
        # Tick type 45 is last timestamp
        if tickType == 45:
            self.live_data[reqId]['last_timestamp'] = value
    
    def tickGeneric(self, reqId, tickType, value):
        """
        Callback for receiving generic tick data.
        
        Args:
            reqId (int): Request identifier
            tickType (int): Type of tick
            value (float): Generic value
        """
        if reqId not in self.live_data:
            self.live_data[reqId] = {'timestamp': datetime.now()}
        
        # Map generic tick types
        generic_map = {
            23: 'option_historical_vol',
            24: 'option_implied_vol',
            31: 'index_future_premium',
            49: 'halted',
            54: 'trade_count',
            55: 'trade_rate',
            56: 'volume_rate',
            58: 'rt_historical_vol'
        }
        
        if tickType in generic_map:
            self.live_data[reqId][generic_map[tickType]] = value
    
    def get_live_data(self, symbol, sec_type='STK', exchange='SMART', 
                     currency='USD', duration=5, generic_tick_list='', snapshot=False):
        """
        Retrieve live streaming market data for a security.
        
        Args:
            symbol (str): Security symbol
            sec_type (str): Security type ('STK', 'OPT', 'FUT', 'CASH', etc.)
            exchange (str): Exchange ('SMART' or specific exchange)
            currency (str): Currency (e.g., 'USD', 'EUR')
            duration (int): How many seconds to collect live data (if not snapshot)
            generic_tick_list (str): Comma-separated generic tick types to request
                                    Common values:
                                    '100,101' - Option volume and open interest
                                    '104,106' - Historical and implied volatility  
                                    '165' - Misc stats (13/26/52 week high/low, avg volume)
                                    '221' - Mark price
                                    '225' - Auction values
                                    '233' - RTVolume (Time & Sales)
                                    '236' - Shortable shares
                                    '258' - Fundamentals
                                    '293,294,295' - Trade count, rate, volume rate
            snapshot (bool): If True, get a single snapshot instead of streaming
            
        Returns:
            dict: Dictionary with current market data
                  Keys: bid, ask, last, high, low, close, volume, bid_size, ask_size, etc.
        
        Examples:
            # Get live stock data for 5 seconds
            data = client.get_live_data('AAPL')
            
            # Get snapshot with additional statistics
            data = client.get_live_data('AAPL', generic_tick_list='165', snapshot=True)
            
            # Get live option data with greeks
            data = client.get_live_data('AAPL', sec_type='OPT', generic_tick_list='104,106')
        """
        if not self.is_connected():
            print("✗ Not connected to IBKR. Call connect() first.")
            return {}
        
        # Create contract
        contract = Contract()
        contract.symbol = symbol
        contract.secType = sec_type
        contract.exchange = exchange
        contract.currency = currency
        
        # Generate request ID
        req_id = self._get_next_req_id()
        
        # Clear previous data
        if req_id in self.live_data:
            del self.live_data[req_id]
        
        print(f"→ Requesting live market data for {symbol}...")
        
        # Request market data
        # Parameters: reqId, contract, genericTickList, snapshot, regulatorySnapshot, mktDataOptions
        self.reqMktData(req_id, contract, generic_tick_list, snapshot, False, [])
        
        if snapshot:
            # For snapshot, wait briefly then cancel
            time.sleep(2)
        else:
            # For streaming, collect for specified duration
            time.sleep(duration)
        
        # Cancel market data subscription
        self.cancelMktData(req_id)
        
        # Return collected data
        result = self.live_data.get(req_id, {})
        if result:
            print(f"✓ Live data retrieved: {len(result)} fields")
        else:
            print("✗ No live data received")
        
        return result
    
    def get_live_data_df(self, symbols, sec_type='STK', exchange='SMART', 
                        currency='USD', generic_tick_list='', snapshot=True):
        """
        Get live market data for multiple symbols and return as DataFrame.
        
        Args:
            symbols (list): List of symbols
            sec_type (str): Security type
            exchange (str): Exchange
            currency (str): Currency
            generic_tick_list (str): Generic ticks to request
            snapshot (bool): Get snapshot (True) or stream briefly (False)
            
        Returns:
            pandas.DataFrame: Market data for all symbols
        
        Example:
            df = client.get_live_data_df(['AAPL', 'MSFT', 'GOOGL'])
        """
        data_list = []
        
        for symbol in symbols:
            data = self.get_live_data(symbol, sec_type, exchange, currency, 
                                     generic_tick_list=generic_tick_list, snapshot=snapshot)
            data['symbol'] = symbol
            data_list.append(data)
        
        return pd.DataFrame(data_list)
    
    # ==================== OPTION CHAIN METHODS ====================
    
    def contractDetails(self, reqId, contractDetails):
        """
        Callback for receiving contract details (used for option chains).
        
        Args:
            reqId (int): Request identifier
            contractDetails (ContractDetails): Detailed contract information
        """
        if reqId not in self.contract_details_data:
            self.contract_details_data[reqId] = []
        
        contract = contractDetails.contract
        
        # Store comprehensive option details
        detail_dict = {
            'symbol': contract.symbol,
            'conId': contract.conId,
            'strike': contract.strike,
            'right': contract.right,  # 'C' for call, 'P' for put
            'expiry': contract.lastTradeDateOrContractMonth,
            'multiplier': contract.multiplier,
            'exchange': contract.exchange,
            'currency': contract.currency,
            'localSymbol': contract.localSymbol,
            'tradingClass': contract.tradingClass,
            # Additional details from contractDetails
            'marketName': contractDetails.marketName,
            'minTick': contractDetails.minTick,
            'priceMagnifier': contractDetails.priceMagnifier,
            'underConId': contractDetails.underConId,
            'longName': contractDetails.longName,
            'contractMonth': contractDetails.contractMonth,
            'industry': contractDetails.industry,
            'category': contractDetails.category,
            'subcategory': contractDetails.subcategory,
            'timeZoneId': contractDetails.timeZoneId,
            'tradingHours': contractDetails.tradingHours,
            'liquidHours': contractDetails.liquidHours
        }
        
        self.contract_details_data[reqId].append(detail_dict)
    
    def contractDetailsEnd(self, reqId):
        """
        Callback when all contract details have been received.
        
        Args:
            reqId (int): Request identifier
        """
        count = len(self.contract_details_data.get(reqId, []))
        print(f"✓ Contract details complete for request {reqId} ({count} contracts)")
        self.contract_details_end_event.set()
    
    def get_option_chain(self, symbol, expiry='', strike=0.0, right='', 
                        exchange='SMART', currency='USD', timeout=60):
        """
        Retrieve option chain for an underlying security.
        
        Args:
            symbol (str): Underlying symbol (e.g., 'AAPL', 'SPY')
            expiry (str): Specific expiration date in 'YYYYMMDD' format (empty for all)
            strike (float): Specific strike price (0.0 for all strikes)
            right (str): Option type - 'C' for calls, 'P' for puts, '' for both
            exchange (str): Exchange (default 'SMART')
            currency (str): Currency (default 'USD')
            timeout (int): Maximum seconds to wait for data
            
        Returns:
            pandas.DataFrame: Option chain with columns including:
                             symbol, conId, strike, right, expiry, multiplier, exchange,
                             currency, localSymbol, tradingClass, minTick, etc.
        
        Examples:
            # Get all options for AAPL
            chain = client.get_option_chain('AAPL')
            
            # Get only calls expiring on a specific date
            chain = client.get_option_chain('AAPL', expiry='20250117', right='C')
            
            # Get specific strike
            chain = client.get_option_chain('AAPL', strike=150.0, right='P')
        
        Note:
            - Requesting entire option chains can be slow due to IBKR throttling
            - Consider using reqSecDefOptParams for faster option chain discovery
            - Specify expiry and/or strike to narrow results and speed up request
        """
        if not self.is_connected():
            print("✗ Not connected to IBKR. Call connect() first.")
            return pd.DataFrame()
        
        # Create option contract
        contract = Contract()
        contract.symbol = symbol
        contract.secType = 'OPT'
        contract.exchange = exchange
        contract.currency = currency
        
        # Apply filters if provided
        if expiry:
            contract.lastTradeDateOrContractMonth = expiry
        if strike > 0:
            contract.strike = strike
        if right:
            contract.right = right
        
        # Generate request ID
        req_id = self._get_next_req_id()
        
        # Clear previous data
        self.contract_details_end_event.clear()
        if req_id in self.contract_details_data:
            del self.contract_details_data[req_id]
        
        filter_str = f" (expiry={expiry}, strike={strike}, right={right})" if expiry or strike or right else ""
        print(f"→ Requesting option chain for {symbol}{filter_str}...")
        
        # Request contract details (option chain)
        self.reqContractDetails(req_id, contract)
        
        # Wait for data
        if self.contract_details_end_event.wait(timeout=timeout):
            if req_id in self.contract_details_data and len(self.contract_details_data[req_id]) > 0:
                df = pd.DataFrame(self.contract_details_data[req_id])
                print(f"✓ Retrieved {len(df)} option contracts")
                
                # Sort by expiry, strike, and right for easier analysis
                if not df.empty:
                    df = df.sort_values(['expiry', 'strike', 'right']).reset_index(drop=True)
                
                return df
            else:
                print("✗ No option contracts found")
                return pd.DataFrame()
        else:
            print(f"✗ Request timeout after {timeout} seconds")
            return pd.DataFrame()
    
    # ==================== OPTION DATA METHODS ====================
    
    def tickOptionComputation(self, reqId, tickType, tickAttrib, impliedVol, delta, 
                             optPrice, pvDividend, gamma, vega, theta, undPrice):
        """
        Callback for receiving option greeks and calculations.
        
        Args:
            reqId (int): Request identifier
            tickType (int): Type of computation (10=bid, 11=ask, 12=last, 13=model)
            tickAttrib (int): Return based (0) or price based (1)
            impliedVol (float): Implied volatility
            delta (float): Option delta
            optPrice (float): Option price
            pvDividend (float): Present value of dividends
            gamma (float): Option gamma
            vega (float): Option vega
            theta (float): Option theta
            undPrice (float): Underlying price
        """
        if reqId not in self.option_greeks:
            self.option_greeks[reqId] = {}
        
        # Map tick types to computation names
        comp_map = {10: 'bid', 11: 'ask', 12: 'last', 13: 'model'}
        comp_name = comp_map.get(tickType, f'type_{tickType}')
        
        self.option_greeks[reqId][f'{comp_name}_computation'] = {
            'impliedVol': impliedVol,
            'delta': delta,
            'optPrice': optPrice,
            'pvDividend': pvDividend,
            'gamma': gamma,
            'vega': vega,
            'theta': theta,
            'undPrice': undPrice,
            'timestamp': datetime.now()
        }
    
    def get_option_data(self, symbol, expiry, strike, right, 
                       exchange='SMART', currency='USD',
                       include_greeks=True, include_historical=False,
                       duration='1 M', bar_size='1 day', timeout=30):
        """
        Get comprehensive data for a specific option contract.
        
        Args:
            symbol (str): Underlying symbol
            expiry (str): Expiration date in 'YYYYMMDD' format
            strike (float): Strike price
            right (str): Option type ('C' for call, 'P' for put)
            exchange (str): Exchange (default 'SMART')
            currency (str): Currency (default 'USD')
            include_greeks (bool): Include option greeks (delta, gamma, vega, theta, IV)
            include_historical (bool): Include historical price data
            duration (str): Duration for historical data (if include_historical=True)
            bar_size (str): Bar size for historical data
            timeout (int): Timeout in seconds
            
        Returns:
            dict: Dictionary containing:
                  - 'contract': Contract details
                  - 'live_data': Current market data (bid, ask, last, volume, etc.)
                  - 'greeks': Option greeks (if include_greeks=True)
                  - 'historical': Historical price data (if include_historical=True)
        
        Examples:
            # Get current option data with greeks
            data = client.get_option_data('AAPL', '20250117', 150.0, 'C')
            
            # Get option data with historical prices
            data = client.get_option_data('AAPL', '20250117', 150.0, 'C',
                                         include_historical=True, duration='1 M')
        """
        if not self.is_connected():
            print("✗ Not connected to IBKR. Call connect() first.")
            return {}
        
        # Create option contract
        contract = Contract()
        contract.symbol = symbol
        contract.secType = 'OPT'
        contract.exchange = exchange
        contract.currency = currency
        contract.lastTradeDateOrContractMonth = expiry
        contract.strike = strike
        contract.right = right
        
        result = {
            'symbol': symbol,
            'expiry': expiry,
            'strike': strike,
            'right': right,
            'contract': None,
            'live_data': {},
            'greeks': {},
            'historical': pd.DataFrame()
        }
        
        print(f"→ Requesting option data for {symbol} {expiry} {strike} {right}...")
        
        # 1. Get contract details
        req_id_contract = self._get_next_req_id()
        self.contract_details_end_event.clear()
        if req_id_contract in self.contract_details_data:
            del self.contract_details_data[req_id_contract]
        
        self.reqContractDetails(req_id_contract, contract)
        
        if self.contract_details_end_event.wait(timeout=timeout):
            if req_id_contract in self.contract_details_data and self.contract_details_data[req_id_contract]:
                result['contract'] = self.contract_details_data[req_id_contract][0]
                print("✓ Contract details retrieved")
        else:
            print("✗ Contract details timeout")
            return result
        
        # 2. Get live market data and greeks
        req_id_live = self._get_next_req_id()
        if req_id_live in self.live_data:
            del self.live_data[req_id_live]
        if req_id_live in self.option_greeks:
            del self.option_greeks[req_id_live]
        
        # Request market data (greeks are automatically included for options)
        self.reqMktData(req_id_live, contract, '', False, False, [])
        time.sleep(3)  # Wait for data
        self.cancelMktData(req_id_live)
        
        result['live_data'] = self.live_data.get(req_id_live, {})
        if include_greeks:
            result['greeks'] = self.option_greeks.get(req_id_live, {})
        
        if result['live_data']:
            print("✓ Live market data retrieved")
        if include_greeks and result['greeks']:
            print("✓ Option greeks retrieved")
        
        # 3. Get historical data if requested
        if include_historical:
            req_id_hist = self._get_next_req_id()
            self.data_received_event.clear()
            if req_id_hist in self.historical_data:
                del self.historical_data[req_id_hist]
            
            self.reqHistoricalData(
                req_id_hist, contract, '', duration, bar_size,
                'TRADES', 1, 1, False, []
            )
            
            if self.data_received_event.wait(timeout=timeout):
                if req_id_hist in self.historical_data:
                    result['historical'] = pd.DataFrame(self.historical_data[req_id_hist])
                    print(f"✓ Historical data retrieved ({len(result['historical'])} bars)")
        
        return result
    
    def get_option_data_df(self, symbol, expiries, strikes, rights, 
                          exchange='SMART', currency='USD'):
        """
        Get option data for multiple contracts and return as DataFrame.
        
        Args:
            symbol (str): Underlying symbol
            expiries (list): List of expiration dates
            strikes (list): List of strike prices
            rights (list): List of option types ('C', 'P', or both)
            exchange (str): Exchange
            currency (str): Currency
            
        Returns:
            pandas.DataFrame: Market data for all option contracts
        
        Example:
            df = client.get_option_data_df('AAPL', 
                                          expiries=['20250117', '20250214'],
                                          strikes=[150, 155, 160],
                                          rights=['C', 'P'])
        """
            
        data_list = []
        contract_num = 0
        
        for expiry in expiries:
            for strike in strikes:
                for right in rights:
                    contract_num += 1
                    
                    
                    data = self.get_option_data(symbol, expiry, strike, right, 
                                               exchange, currency, 
                                               include_greeks=True, 
                                               include_historical=False)
                    
                    # Flatten data for DataFrame
                    row = {
                        'symbol': symbol,
                        'expiry': expiry,
                        'strike': strike,
                        'right': right
                    }
                    row.update(data.get('live_data', {}))
                    
                    # Add greeks if available
                    if 'greeks' in data and data['greeks']:
                        for comp_type, greeks in data['greeks'].items():
                            if isinstance(greeks, dict):
                                for key, val in greeks.items():
                                    if key != 'timestamp':
                                        row[f'{comp_type}_{key}'] = val
                    
                    data_list.append(row)
        
        # Create DataFrame
        df = pd.DataFrame(data_list)
        
        # Display summary
        print(f"\n\n✓ Successfully retrieved data for {len(df)} contracts")
        
        if not df.empty:
            print(f"\nDataFrame Shape: {df.shape[0]} rows × {df.shape[1]} columns")
            print(f"\nColumns: {', '.join(df.columns.tolist())}")
            print(f"\n{'='*80}")
            print("Option Data Summary:")
            print(f"{'='*80}")
            print(df.to_string())
            print(f"{'='*80}\n")
        
        return df
    
    # ==================== UTILITY METHODS ====================
    
    def __enter__(self):
        """Context manager entry - connect automatically."""
        self.connect()
        return self
    
    def __exit__(self, exc_type, exc_val, exc_tb):
        """Context manager exit - disconnect automatically."""
        self.disconnect()
    
    def __del__(self):
        """Destructor - ensure disconnection."""
        if self.is_connected():
            self.disconnect()

## Usage Examples

Below are comprehensive examples demonstrating how to use the `IBKR_Functions` class.

In [2]:
# ==================== EXAMPLE 1: Basic Connection and Historical Data ====================

def example_1_basic_historical():
    """
    Example 1: Connect to IBKR and retrieve historical stock data.
    """
    print("="*80)
    print("EXAMPLE 1: Basic Historical Data Retrieval")
    print("="*80)
    
    # Create client instance
    client = IBKR_Functions(host='127.0.0.1', port=7496, client_id=1)
    
    # Connect to IBKR
    if client.connect():
        time.sleep(1)
        
        # Get 1 year of daily data for Apple stock
        print("\n--- Retrieving AAPL Daily Data (1 Year) ---")
        df_daily = client.get_historical_data(
            symbol='AAPL',
            duration='1 Y',
            bar_size='1 day',
            what_to_show='TRADES'
        )
        
        if not df_daily.empty:
            print(f"\nRetrieved {len(df_daily)} daily bars")
            print("\nFirst 5 rows:")
            print(df_daily.head())
            print("\nLast 5 rows:")
            print(df_daily.tail())
            
            # Basic statistics
            print("\nBasic Statistics:")
            print(f"Highest Close: ${df_daily['close'].max():.2f}")
            print(f"Lowest Close: ${df_daily['close'].min():.2f}")
            print(f"Average Close: ${df_daily['close'].mean():.2f}")
            print(f"Total Volume: {df_daily['volume'].sum():,.0f}")
        
        # Get 1 week of hourly data
        print("\n--- Retrieving AAPL Hourly Data (1 Week) ---")
        df_hourly = client.get_historical_data(
            symbol='AAPL',
            duration='1 W',
            bar_size='1 hour',
            what_to_show='TRADES',
            use_rth=True  # Regular trading hours only
        )
        
        if not df_hourly.empty:
            print(f"\nRetrieved {len(df_hourly)} hourly bars")
            print(df_hourly.head(10))
        
        # Disconnect
        client.disconnect()
    else:
        print("Failed to connect to IBKR")

# Uncomment to run:
# example_1_basic_historical()


# ==================== EXAMPLE 2: Live Market Data ====================

def example_2_live_data():
    """
    Example 2: Get live streaming market data for multiple stocks.
    """
    print("="*80)
    print("EXAMPLE 2: Live Market Data Streaming")
    print("="*80)
    
    client = IBKR_Functions(host='127.0.0.1', port=7496, client_id=2)
    
    if client.connect():
        time.sleep(1)
        
        # Get live data for a single stock
        print("\n--- Single Stock Live Data ---")
        live_data = client.get_live_data(
            symbol='AAPL',
            duration=5,  # Collect for 5 seconds
            generic_tick_list='165'  # Include 52-week high/low, avg volume
        )
        
        print("\nLive Data for AAPL:")
        for key, value in live_data.items():
            print(f"  {key}: {value}")
        
        # Get snapshot data for multiple stocks
        print("\n--- Multiple Stocks Snapshot ---")
        symbols = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'TSLA']
        df_live = client.get_live_data_df(
            symbols=symbols,
            snapshot=True,
            generic_tick_list='165'
        )
        
        print("\nLive Market Data DataFrame:")
        print(df_live[['symbol', 'last', 'bid', 'ask', 'volume', 'high', 'low']])
        
        client.disconnect()
    else:
        print("Failed to connect to IBKR")

# Uncomment to run:
# example_2_live_data()


# ==================== EXAMPLE 3: Option Chain Analysis ====================

def example_3_option_chain():
    """
    Example 3: Retrieve and analyze option chains.
    """
    print("="*80)
    print("EXAMPLE 3: Option Chain Analysis")
    print("="*80)
    
    client = IBKR_Functions(host='127.0.0.1', port=7496, client_id=3)
    
    if client.connect():
        time.sleep(1)
        
        # Get all call options for AAPL
        print("\n--- Retrieving AAPL Call Options ---")
        calls = client.get_option_chain(
            symbol='AAPL',
            right='C'  # Calls only
        )
        
        if not calls.empty:
            print(f"\nFound {len(calls)} call option contracts")
            
            # Show unique expiration dates
            unique_expiries = sorted(calls['expiry'].unique())
            print(f"\nAvailable Expirations ({len(unique_expiries)}):")
            for i, exp in enumerate(unique_expiries[:10], 1):  # Show first 10
                print(f"  {i}. {exp}")
            
            # Analyze a specific expiration
            if len(unique_expiries) > 0:
                target_expiry = unique_expiries[0]  # First expiration
                print(f"\n--- Analyzing Expiry: {target_expiry} ---")
                
                expiry_calls = calls[calls['expiry'] == target_expiry]
                print(f"\nStrikes available: {len(expiry_calls)}")
                print("\nStrike prices:")
                print(expiry_calls[['strike', 'localSymbol']].head(15))
        
        # Get put options for a specific expiry and strike range
        print("\n--- Retrieving Filtered Put Options ---")
        puts = client.get_option_chain(
            symbol='AAPL',
            expiry='20250117',  # January 2025 expiry (adjust as needed)
            right='P'  # Puts only
        )
        
        if not puts.empty:
            print(f"\nFound {len(puts)} put option contracts")
            print(puts[['strike', 'localSymbol', 'tradingClass']].head(10))
        
        client.disconnect()
    else:
        print("Failed to connect to IBKR")

# Uncomment to run:
# example_3_option_chain()


# ==================== EXAMPLE 4: Specific Option Data with Greeks ====================

def example_4_option_data_greeks():
    """
    Example 4: Get detailed data for specific option contracts including greeks.
    """
    print("="*80)
    print("EXAMPLE 4: Option Data with Greeks")
    print("="*80)
    
    client = IBKR_Functions(host='127.0.0.1', port=7496, client_id=4)
    
    if client.connect():
        time.sleep(1)
        
        # Define option contract
        symbol = 'AAPL'
        expiry = '20250117'  # Adjust to valid expiry
        strike = 150.0       # Adjust to valid strike
        right = 'C'          # Call option
        
        print(f"\n--- Retrieving Data for {symbol} {expiry} {strike} {right} ---")
        
        # Get comprehensive option data
        option_data = client.get_option_data(
            symbol=symbol,
            expiry=expiry,
            strike=strike,
            right=right,
            include_greeks=True,
            include_historical=False
        )
        
        # Display contract details
        if option_data['contract']:
            print("\nContract Details:")
            contract = option_data['contract']
            print(f"  Symbol: {contract['symbol']}")
            print(f"  Local Symbol: {contract['localSymbol']}")
            print(f"  Strike: ${contract['strike']}")
            print(f"  Expiry: {contract['expiry']}")
            print(f"  Multiplier: {contract['multiplier']}")
            print(f"  Exchange: {contract['exchange']}")
            print(f"  Min Tick: {contract['minTick']}")
        
        # Display live market data
        if option_data['live_data']:
            print("\nLive Market Data:")
            live = option_data['live_data']
            for key in ['bid', 'ask', 'last', 'volume', 'bid_size', 'ask_size']:
                if key in live:
                    print(f"  {key}: {live[key]}")
        
        # Display option greeks
        if option_data['greeks']:
            print("\nOption Greeks:")
            greeks = option_data['greeks']
            for comp_type, values in greeks.items():
                print(f"\n  {comp_type}:")
                if isinstance(values, dict):
                    for key, val in values.items():
                        if key != 'timestamp' and val is not None:
                            print(f"    {key}: {val}")
        
        client.disconnect()
    else:
        print("Failed to connect to IBKR")

# Uncomment to run:
# example_4_option_data_greeks()


# ==================== EXAMPLE 5: Option Historical Data ====================

def example_5_option_historical():
    """
    Example 5: Get historical price data for option contracts.
    """
    print("="*80)
    print("EXAMPLE 5: Option Historical Data")
    print("="*80)
    
    client = IBKR_Functions(host='127.0.0.1', port=7496, client_id=5)
    
    if client.connect():
        time.sleep(1)
        
        # Get option data with historical prices
        symbol = 'AAPL'
        expiry = '20250117'
        strike = 150.0
        right = 'C'
        
        print(f"\n--- Retrieving Historical Data for {symbol} {expiry} {strike} {right} ---")
        
        option_data = client.get_option_data(
            symbol=symbol,
            expiry=expiry,
            strike=strike,
            right=right,
            include_greeks=True,
            include_historical=True,
            duration='1 M',
            bar_size='1 day'
        )
        
        # Display historical data
        if not option_data['historical'].empty:
            df_hist = option_data['historical']
            print(f"\nRetrieved {len(df_hist)} historical bars")
            print("\nFirst 5 bars:")
            print(df_hist.head())
            print("\nLast 5 bars:")
            print(df_hist.tail())
            
            # Calculate some metrics
            print("\nOption Price Statistics:")
            print(f"  Highest Close: ${df_hist['close'].max():.2f}")
            print(f"  Lowest Close: ${df_hist['close'].min():.2f}")
            print(f"  Average Close: ${df_hist['close'].mean():.2f}")
            print(f"  Latest Close: ${df_hist['close'].iloc[-1]:.2f}")
        
        client.disconnect()
    else:
        print("Failed to connect to IBKR")

# Uncomment to run:
# example_5_option_historical()


# ==================== EXAMPLE 6: Context Manager Usage ====================

def example_6_context_manager():
    """
    Example 6: Using IBKR_Functions with context manager for automatic connection/disconnection.
    """
    print("="*80)
    print("EXAMPLE 6: Context Manager Usage")
    print("="*80)
    
    # Use 'with' statement for automatic connection and disconnection
    with IBKR_Functions(host='127.0.0.1', port=7496, client_id=6) as client:
        time.sleep(1)
        
        print("\n--- Getting data using context manager ---")
        
        # Get historical data
        df = client.get_historical_data('MSFT', duration='1 M', bar_size='1 day')
        
        if not df.empty:
            print(f"\nRetrieved {len(df)} bars for MSFT")
            print(df.tail())
        
        # Get live data
        live = client.get_live_data('MSFT', duration=3, snapshot=True)
        print(f"\nLive data for MSFT: {live}")
    
    # Connection automatically closed after 'with' block
    print("\n✓ Connection automatically closed")

# Uncomment to run:
# example_6_context_manager()


# ==================== EXAMPLE 7: Multiple Option Contracts ====================

def example_7_multiple_options():
    """
    Example 7: Get data for multiple option contracts efficiently.
    """
    print("="*80)
    print("EXAMPLE 7: Multiple Option Contracts")
    print("="*80)
    
    client = IBKR_Functions(host='127.0.0.1', port=7496, client_id=7)
    
    if client.connect():
        time.sleep(1)
        
        # Define multiple contracts
        symbol = 'AAPL'
        expiries = ['20250117']  # Can add more expiries
        strikes = [145.0, 150.0, 155.0, 160.0]  # Multiple strikes
        rights = ['C', 'P']  # Both calls and puts
        
        print(f"\n--- Retrieving data for {len(strikes) * len(rights)} option contracts ---")
        
        # Get data for all combinations
        df_options = client.get_option_data_df(
            symbol=symbol,
            expiries=expiries,
            strikes=strikes,
            rights=rights
        )
        
        if not df_options.empty:
            print(f"\nRetrieved data for {len(df_options)} option contracts")
            print("\nOption Data Summary:")
            print(df_options[['strike', 'right', 'bid', 'ask', 'last', 'volume']])
            
            # Analyze calls vs puts
            calls = df_options[df_options['right'] == 'C']
            puts = df_options[df_options['right'] == 'P']
            
            print(f"\nCalls: {len(calls)}, Puts: {len(puts)}")
        
        client.disconnect()
    else:
        print("Failed to connect to IBKR")

# Uncomment to run:
# example_7_multiple_options()


# ==================== EXAMPLE 8: European Stock (BNP Example) ====================

def example_8_european_stock():
    """
    Example 8: Working with European stocks and options (e.g., BNP).
    """
    print("="*80)
    print("EXAMPLE 8: European Stock - BNP")
    print("="*80)
    
    client = IBKR_Functions(host='127.0.0.1', port=7496, client_id=8)
    
    if client.connect():
        time.sleep(1)
        
        # Get historical data for BNP (French stock)
        print("\n--- BNP Historical Data ---")
        df_bnp = client.get_historical_data(
            symbol='BNP',
            duration='1 M',
            bar_size='1 day',
            sec_type='STK',
            exchange='SBF',  # Euronext Paris
            currency='EUR'
        )
        
        if not df_bnp.empty:
            print(f"\nRetrieved {len(df_bnp)} daily bars for BNP")
            print(df_bnp.tail(10))
        
        # Get live data for BNP
        print("\n--- BNP Live Data ---")
        live_bnp = client.get_live_data(
            symbol='BNP',
            sec_type='STK',
            exchange='SBF',
            currency='EUR',
            duration=5
        )
        print(f"\nLive data: {live_bnp}")
        
        # Get BNP option chain
        print("\n--- BNP Option Chain ---")
        bnp_options = client.get_option_chain(
            symbol='BNP',
            exchange='EUREX',
            currency='EUR',
            right='C'  # Calls
        )
        
        if not bnp_options.empty:
            print(f"\nFound {len(bnp_options)} BNP call options")
            print(bnp_options[['expiry', 'strike', 'localSymbol']].head(15))
        
        client.disconnect()
    else:
        print("Failed to connect to IBKR")

# Uncomment to run:
# example_8_european_stock()


# ==================== MASTER EXAMPLE RUNNER ====================

def run_all_examples():
    """
    Run all examples sequentially.
    WARNING: This will take several minutes to complete.
    """
    print("\n" + "="*80)
    print("RUNNING ALL EXAMPLES")
    print("="*80 + "\n")
    
    examples = [
        ("Example 1: Basic Historical Data", example_1_basic_historical),
        ("Example 2: Live Market Data", example_2_live_data),
        ("Example 3: Option Chain", example_3_option_chain),
        ("Example 4: Option Greeks", example_4_option_data_greeks),
        ("Example 5: Option Historical", example_5_option_historical),
        ("Example 6: Context Manager", example_6_context_manager),
        ("Example 7: Multiple Options", example_7_multiple_options),
        ("Example 8: European Stock", example_8_european_stock),
    ]
    
    for i, (name, func) in enumerate(examples, 1):
        print(f"\n{'='*80}")
        print(f"Running {name}")
        print(f"{'='*80}\n")
        
        try:
            func()
            print(f"\n✓ {name} completed successfully")
        except Exception as e:
            print(f"\n✗ {name} failed with error: {str(e)}")
        
        # Pause between examples
        if i < len(examples):
            print("\nWaiting 2 seconds before next example...")
            time.sleep(2)
    
    print("\n" + "="*80)
    print("ALL EXAMPLES COMPLETED")
    print("="*80)

# Uncomment to run all examples:
# run_all_examples()

# Or run individual examples:
# example_1_basic_historical()
# example_2_live_data()
# example_3_option_chain()
# example_4_option_data_greeks()
# example_5_option_historical()
# example_6_context_manager()
# example_7_multiple_options()
# example_8_european_stock()

## Quick Reference Guide

### Class: `IBKR_Functions`

A comprehensive class for interacting with Interactive Brokers API to retrieve:
- **Live market data** (streaming or snapshot)
- **Historical data** (bars with flexible time ranges and granularity)
- **Option chains** (all or filtered option contracts)
- **Option data** (specific contracts with greeks and historical prices)

---

### Connection Methods

```python
client = IBKR_Functions(host='127.0.0.1', port=7496, client_id=1)
client.connect()           # Connect to IBKR
client.is_connected()      # Check connection status
client.disconnect()        # Disconnect from IBKR
```

**Port Numbers:**
- `7497` - TWS Paper Trading
- `7496` - TWS Live Trading  
- `4001` - IB Gateway Paper
- `4000` - IB Gateway Live

---

### Historical Data

```python
df = client.get_historical_data(
    symbol='AAPL',
    duration='1 Y',        # '1 D', '1 W', '1 M', '1 Y', etc.
    bar_size='1 day',      # '1 min', '5 mins', '1 hour', '1 day', etc.
    what_to_show='TRADES', # 'TRADES', 'MIDPOINT', 'BID', 'ASK', 'ADJUSTED_LAST'
    sec_type='STK',        # 'STK', 'OPT', 'FUT', 'CASH', 'IND'
    exchange='SMART',
    currency='USD',
    use_rth=True           # True=regular hours, False=extended hours
)
```

**Returns:** pandas.DataFrame with columns: `date, open, high, low, close, volume, average, barCount`

---

### Live Market Data

```python
# Single symbol
data = client.get_live_data(
    symbol='AAPL',
    duration=5,            # Seconds to collect data
    snapshot=False,        # True for single snapshot, False for streaming
    generic_tick_list=''   # Additional data: '165', '100,101', '104,106', etc.
)

# Multiple symbols as DataFrame
df = client.get_live_data_df(
    symbols=['AAPL', 'MSFT', 'GOOGL'],
    snapshot=True
)
```

**Returns:** Dictionary or DataFrame with: `bid, ask, last, volume, high, low, close, etc.`

---

### Option Chain

```python
df = client.get_option_chain(
    symbol='AAPL',
    expiry='',             # Empty for all, or 'YYYYMMDD' for specific
    strike=0.0,            # 0.0 for all, or specific strike price
    right='',              # '' for both, 'C' for calls, 'P' for puts
    exchange='SMART',
    currency='USD'
)
```

**Returns:** pandas.DataFrame with option contracts including: `strike, right, expiry, conId, multiplier, localSymbol, etc.`

---

### Specific Option Data

```python
data = client.get_option_data(
    symbol='AAPL',
    expiry='20250117',     # Format: YYYYMMDD
    strike=150.0,
    right='C',             # 'C' for call, 'P' for put
    include_greeks=True,   # Include delta, gamma, theta, vega, IV
    include_historical=True, # Include historical price data
    duration='1 M',
    bar_size='1 day'
)
```

**Returns:** Dictionary with:
- `contract`: Contract details
- `live_data`: Current market data
- `greeks`: Option greeks (delta, gamma, vega, theta, implied volatility)
- `historical`: Historical price DataFrame

---

### Multiple Option Contracts

```python
df = client.get_option_data_df(
    symbol='AAPL',
    expiries=['20250117', '20250214'],
    strikes=[145.0, 150.0, 155.0],
    rights=['C', 'P']
)
```

**Returns:** pandas.DataFrame with market data for all combinations of expiries, strikes, and rights

---

### Generic Tick Lists (for Live Data)

Add to `generic_tick_list` parameter for additional data:

- `'100,101'` - Option volume and open interest
- `'104,106'` - Historical and implied volatility
- `'165'` - 13/26/52 week highs/lows, average volume
- `'221'` - Mark price
- `'233'` - RTVolume (Time & Sales)
- `'236'` - Shortable shares
- `'293,294,295'` - Trade count, rate, volume rate

---

### Bar Sizes

Available `bar_size` values:
- Seconds: `'1 secs', '5 secs', '10 secs', '15 secs', '30 secs'`
- Minutes: `'1 min', '2 mins', '3 mins', '5 mins', '10 mins', '15 mins', '20 mins', '30 mins'`
- Hours: `'1 hour', '2 hours', '3 hours', '4 hours', '8 hours'`
- Days: `'1 day', '1 week', '1 month'`

---

### Duration Strings

Format: `'<number> <unit>'`

Units: `S` (seconds), `D` (days), `W` (weeks), `M` (months), `Y` (years)

Examples: `'1 Y'`, `'6 M'`, `'1 W'`, `'3 D'`, `'86400 S'`

---

### Security Types

- `'STK'` - Stocks
- `'OPT'` - Options
- `'FUT'` - Futures
- `'CASH'` - Forex
- `'IND'` - Indices
- `'CFD'` - CFDs
- `'BOND'` - Bonds

---

### Common Exchanges

**US Stocks:**
- `'SMART'` - Smart routing (recommended)
- `'NYSE'`, `'NASDAQ'`, `'ARCA'`

**European Stocks:**
- `'SBF'` - Euronext Paris
- `'LSE'` - London Stock Exchange
- `'IBIS'` - Deutsche Börse

**European Options:**
- `'EUREX'` - Eurex (Germany)
- `'LIFFE'` - Euronext (London)

**Forex:**
- `'IDEALPRO'` - For forex pairs

---

### Context Manager (Recommended)

```python
# Automatic connection and disconnection
with IBKR_Functions(host='127.0.0.1', port=7496, client_id=1) as client:
    df = client.get_historical_data('AAPL', duration='1 M', bar_size='1 day')
    # Connection automatically closed after this block
```

---

### Important Notes

1. **Connection Required**: Always call `connect()` before making requests
2. **Rate Limits**: IBKR has pacing limitations - avoid making too many requests too quickly
3. **Market Data Subscriptions**: You need appropriate subscriptions for different data types
4. **Time Zones**: Dates use the timezone set in TWS/Gateway login settings
5. **Client IDs**: Use unique client IDs (0-32) for different connections
6. **Regular vs Extended Hours**: Set `use_rth=False` to include pre/post market data
7. **Error Handling**: Check for empty DataFrames/dictionaries to handle errors gracefully

---

### Troubleshooting

**Connection Issues:**
- Ensure TWS or IB Gateway is running
- Check that API connections are enabled in TWS settings
- Verify correct port number
- Check that client ID is not already in use

**No Data Received:**
- Verify you have appropriate market data subscriptions
- Check contract details are correct (symbol, exchange, currency)
- Ensure markets are open (or use historical data)
- Check IBKR error messages in output

**Slow Performance:**
- Use specific expiry/strike filters for option chains
- Reduce number of bars requested for historical data
- Use snapshots instead of streaming for quick queries

In [3]:
# Simple example
client = IBKR_Functions(host='127.0.0.1', port=7496, client_id=6)
client.disconnect()
client.connect()

# Get historical data
df = client.get_historical_data('BNP', duration='1 M', bar_size='1 day', exchange='SBF', currency='EUR')

# Get live data
#live = client.get_live_data('AAPL', duration=5)

# Get option chain
#options = client.get_option_chain('AAPL', right='C')

# Get specific option with greeks
#Option_Data = client.get_option_data('BNP', '20260116', 80.0, 'C', exchange='EUREX', currency='EUR',include_greeks=True)
print(df.head())
client.disconnect()

✓ Disconnected from IBKR
ℹ Connection established - Next valid order ID: 1
✓ Successfully connected to IBKR on 127.0.0.1:7496 (Client ID: 6)
ℹ Info: Market data farm connection is OK:eufarmnj
ℹ Info: Market data farm connection is OK:cashfarm
ℹ Info: Market data farm connection is OK:usopt.nj
ℹ Info: Market data farm connection is OK:usfarm.nj
ℹ Info: Market data farm connection is OK:eufarm
ℹ Info: Market data farm connection is OK:usopt
ℹ Info: Market data farm connection is OK:usfarm
ℹ Info: HMDS data farm connection is OK:euhmds
ℹ Info: HMDS data farm connection is OK:fundfarm
ℹ Info: HMDS data farm connection is OK:ushmds
ℹ Info: Sec-def data farm connection is OK:secdefil
→ Requesting historical data for BNP...
✓ Historical data received for request 1001 (20251124  21:34:29 to 20251224  21:34:29)
✓ Retrieved 22 bars of historical data
       date   open   high    low  close   volume  average  barCount
0  20251125  70.45  72.25  70.05  71.77  3301681  71.5915     20155
1  20251126

✓ Disconnected from IBKR
